***

# Анализ данных сервиса о книгах

## Задача

 - Провести исследовательский анализ данных сервиса-конкурента, благодаря которому станет возможно сформулировать ценностное предложение для нового продукта

 ## План


- 1. Подготовка данных  
   - Подключение к базе данных  
   - Изучение таблиц базы данных  
  
- 2. Анализ данных  
   - I. количество книг, выпущенных после 1 января 2000 года  
   - II. количество пользовательских обзоров и средняя оценка для каждой книги  
   - III. издательство, которое издало наибольшее число книг толще 50 страниц  
   - VI. автор с самой высокой средней оценкой книг  
   - V. среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок  
  
- 3. Выводы  

***

# 1. Подготовка данных

## Подключение к базе данных

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
db_config = {'user': 'praktikum_student',
             'pwd': 'Sdf4$2;d-d30pp',
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, 
             'db': 'data-analyst-final-project-db'} 

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])            

engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [3]:
def SQL_request_result(query):
    return pd.io.sql.read_sql(query, con = engine)

## Изучение таблиц базы данных

### books

In [4]:
query = """SELECT *
           FROM books
           LIMIT 1;
        """

SQL_request_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93


**Описание данных**

`book_id` — идентификатор книги  
`author_id` — идентификатор автора  
`title` — название книги  
`num_pages` — количество страниц  
`publication_date` — дата публикации книги  
`publisher_id` — идентификатор издателя  

### authors

In [5]:
query = """SELECT *
           FROM authors
           LIMIT 1;
        """

SQL_request_result(query)

,author_id,author
0,1,A.S. Byatt


**Описание данных**  

`author_id` — идентификатор автора  
`author` — имя автора  

### publishers

In [6]:
query = """SELECT *
           FROM publishers
           LIMIT 1;
        """

SQL_request_result(query)

,publisher_id,publisher
0,1,Ace


**Описание данных**  

`publisher_id` — идентификатор издательства  
`publisher` — название издательства  

### ratings

In [7]:
query = """SELECT *
           FROM ratings
           LIMIT 1;
        """

SQL_request_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4


**Описание данных**  

`rating_id` — идентификатор оценки  
`book_id` — идентификатор книги  
`username` — имя пользователя, оставившего оценку  
`rating` — оценка книги  

### reviews

In [8]:
query = """SELECT *
           FROM reviews
           LIMIT 1;
        """

SQL_request_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...


**Описание данных**  

`review_id` — идентификатор обзора  
`book_id` — идентификатор книги  
`username` — имя пользователя, написавшего обзор  
`text` — текст обзора  

# 2. Анализ данных

## I. Количество книг, выпущенных после 1 января 2000 года

In [9]:
query = """SELECT COUNT(book_id) AS amount_books
           FROM books
           WHERE publication_date > '2000-01-01';
        """

SQL_request_result(query)

,amount_books
0,819


 - количество книг, выпущенных после 1 января 2000 года - 819

## II. Количество пользовательских обзоров и средняя оценка для каждой книги

#### Количество обзоров каждой книги

In [10]:
query = """SELECT book_id, COUNT(review_id) as number_of_reviews
           FROM reviews
           GROUP BY book_id;
        """

SQL_request_result(query)

,book_id,number_of_reviews
0,652,2
1,273,2
2,51,5
3,951,2
4,839,4
...,...,...
989,64,4
990,55,2
991,148,3
992,790,2


#### Средний рейтинг каждой книги

In [11]:
query = """SELECT book_id, AVG(rating) as average_rating
           FROM ratings
           GROUP BY book_id;
        """

SQL_request_result(query)

,book_id,average_rating
0,652,4.500000
1,273,4.500000
2,51,4.250000
3,951,4.000000
4,839,4.285714
...,...,...
995,64,4.230769
996,55,5.000000
997,148,3.428571
998,790,3.500000


#### Количество пользовательских обзоров и средняя оценка для каждой книги, выведим первые топ-10 книг

In [12]:
query = """SELECT books.title AS title, SUBQ1.number_of_reviews, ROUND(SUBQ2.average_rating, 2) AS average_rating
           FROM books
           INNER JOIN 
                (SELECT book_id, COUNT(review_id) as number_of_reviews
                 FROM reviews
                 GROUP BY book_id) AS SUBQ1 ON SUBQ1.book_id = books.book_id
           INNER JOIN 
                (SELECT book_id, AVG(rating) as average_rating
                 FROM ratings
                 GROUP BY book_id) AS SUBQ2 ON SUBQ2.book_id = books.book_id
           GROUP BY title, SUBQ1.number_of_reviews, average_rating
           ORDER BY SUBQ1.number_of_reviews DESC
           LIMIT 10;
        """

SQL_request_result(query)

,title,number_of_reviews,average_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,The Da Vinci Code (Robert Langdon #2),6,3.83
3,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
4,Outlander (Outlander #1),6,4.13
5,Eat Pray Love,6,3.40
6,Water for Elephants,6,3.98
7,The Hobbit or There and Back Again,6,4.13
8,The Book Thief,6,4.26
9,The Alchemist,6,3.79


- наибольшее количество отзывов не влечет за собой высокие оценки для книг  
- по отзывам лидируют "Сумерки"  
- в топ-10 книг по количеству отзывов преобладает фантастика

## III. Издательство, которое издало наибольшее число книг толще 50 страниц

#### Книги толще 50 страниц

In [13]:
query = """SELECT book_id, title, publisher_id
           FROM books
           WHERE num_pages > 50;
        """

SQL_request_result(query)

,book_id,title,publisher_id
0,1,'Salem's Lot,93
1,2,1 000 Places to See Before You Die,336
2,3,13 Little Blue Envelopes (Little Blue Envelope...,135
3,4,1491: New Revelations of the Americas Before C...,309
4,5,1776,268
...,...,...,...
987,996,Wyrd Sisters (Discworld #6; Witches #2),147
988,997,Xenocide (Ender's Saga #3),297
989,998,Year of Wonders,212
990,999,You Suck (A Love Story #2),331


#### Издательства и количество изданых ими книг, которые толще 50 страниц

In [14]:
query = """SELECT publishers.publisher, COUNT(Q1.book_id) AS count_book
           FROM publishers 
           INNER JOIN 
               (SELECT book_id, title, publisher_id
                FROM books
                WHERE num_pages > 50) AS Q1 ON Q1.publisher_id = publishers.publisher_id
           GROUP BY publishers.publisher, publishers.publisher_id
           ORDER BY count_book DESC
           ;
        """

SQL_request_result(query)

,publisher,count_book
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Ballantine Books (NY),1
330,Plaza y Janés,1
331,HarperCollinsPublishers,1
332,Random House: Modern Library,1


#### Максимальное количество изданных книг более 50 страниц

In [15]:
query = """SELECT  MAX(count_book) AS max_count
           FROM 
               (SELECT publishers.publisher, COUNT(Q1.book_id) AS count_book
                FROM publishers 
                INNER JOIN 
                   (SELECT book_id, title, publisher_id
                    FROM books
                    WHERE num_pages > 50) AS Q1 ON Q1.publisher_id = publishers.publisher_id
                GROUP BY publishers.publisher, publishers.publisher_id
                ORDER BY count_book DESC) AS Q2;
        """

SQL_request_result(query)

,max_count
0,42


#### Издательство, которое издало наибольшее число книг толще 50 страниц

In [16]:
query = """SELECT publishers.publisher, COUNT(Q1.book_id) AS max_count
           FROM publishers 
           INNER JOIN 
               (SELECT book_id, title, publisher_id
                FROM books
                WHERE num_pages > 50) AS Q1 ON Q1.publisher_id = publishers.publisher_id
           GROUP BY publishers.publisher
           HAVING COUNT(Q1.book_id) IN 
                                  (SELECT  MAX(count_book)
                                   FROM 
                                       (SELECT publishers.publisher, COUNT(Q1.book_id) AS count_book
                                        FROM publishers 
                                        INNER JOIN 
                                           (SELECT book_id, title, publisher_id
                                            FROM books
                                            WHERE num_pages > 50) AS Q1 ON Q1.publisher_id = publishers.publisher_id
                                    GROUP BY publishers.publisher, publishers.publisher_id
                                    ORDER BY count_book DESC) AS Q2);
        """

SQL_request_result(query)

,publisher,max_count
0,Penguin Books,42


- в базе данных самое часто встречающееся издательство - `"Penguin Books"`

## IV. Автор с самой высокой средней оценкой книг

Учитываются только книги с 50 и более пользовательскими оценками

In [71]:
query = """SELECT authors.author,  ROUND(AVG(ratings.rating),2) AS avg_rate
           FROM authors
           JOIN books USING(author_id)
           JOIN ratings USING(book_id)
           GROUP BY authors.author
           HAVING COUNT(ratings.rating) > 50
           ORDER BY AVG(ratings.rating) DESC
           LIMIT 1;
        """

SQL_request_result(query)

,author,avg_rate
0,J.K. Rowling/Mary GrandPré,4.29


- наибольшая средняя оценка у автора книг о Гарри Поттере.

## V. Среднее количество текстовых обзоров пользователей, которые поставили более чем по 50 оценок

Пользователи поставившие более 50 оценок

In [68]:
query = """SELECT username, COUNT(rating)
           FROM ratings 
           GROUP BY username
           HAVING COUNT(rating) > 50;
        """

SQL_request_result(query)

,username,count
0,sfitzgerald,55
1,jennifermiller,53
2,xdavis,51
3,paul88,56
4,martinadam,56
5,richard89,55


Количество рецензий пользователей, которые имеют более 50 оценок

In [64]:
query = """SELECT username, COUNT(review_id)
           FROM reviews 
           WHERE username IN 
                          (SELECT username
                           FROM ratings 
                           GROUP BY username
                           HAVING COUNT(rating) > 50) 
           GROUP BY username;
        """

SQL_request_result(query)

,username,count
0,sfitzgerald,28
1,jennifermiller,25
2,xdavis,18
3,paul88,22
4,martinadam,27
5,richard89,26


Среднее количество рецензий пользователей, которые имеют более 50 оценок

In [69]:
query = """SELECT ROUND(AVG(count)) AS average_reviews
           FROM 
              (SELECT username, COUNT(review_id)
               FROM reviews 
               WHERE username IN 
                              (SELECT username
                               FROM ratings 
                               GROUP BY username
                               HAVING COUNT(rating) > 50) 
               GROUP BY username) AS Q1
        """

SQL_request_result(query)

,average_reviews
0,24.0


 - 24 обзора на пользователя в среднем - достаточно небольшое количество. Возможно стоит предусмотреть систему поощрения пользователей за обзоры. Это поможет остальным пользователям подбирать интересные для себя книги

# 3.Вывод

 - **Количество книг младше 2000 года очень мало, на него не стоит ориентироваться.** 
 - **В топ-10 самых оцениваемых книг входит преимущественно фэнтези.**  
 - **Наивысшая средняя оценка у автора книги в жанре фэнтези.**    
 - **Стоит предусмотреть систему вознаграждений пользователей за оставленные отзывы о книгах.**